<!-- @format -->

#### Do all the imports


In [1]:
import requests   # for getting the web data
from dotenv import load_dotenv   # for dealing with secrets and environment variables
from os import getenv  # more environment variable stuff
from json import dump  # 

<!-- @format -->

#### load the environments


In [2]:
load_dotenv("techtalk_SEC.secrets")   # userids and passwords and api keys and so forth
load_dotenv("techtalk_SEC.env")    # miscellaneous setup stuff that isn't actually secret

True

<!-- @format -->

#### put all the pieces of the api call together


In [3]:
r = requests.Request(
    method="GET",
    url=f'{getenv("WEB_URL")}/files/company_tickers.json',
    headers={
        "User-Agent": getenv("SEC-User-Agent"),        # We hide the user agent so that bots don't scrape and misuse it
        "Accept": "application/json",
    },
)



<!-- @format -->

#### Go get the data


In [4]:
s = requests.Session()

req = s.send(r.prepare())

<!-- @format -->

#### Save the response to an external file (completely optional but nice to have)


In [5]:
with open(file="cik_tickers.json", mode="w") as file:
    dump(req.json(), file)

#### stash the ticker mapping into a dict for later


In [6]:
the_json = req.json()
ticker_cik = {the_json[a]["ticker"]: str(the_json[a]["cik_str"]).zfill(10) for a in the_json}
cik_ticker = {str(the_json[a]["cik_str"]).zfill(10): the_json[a]["ticker"] for a in the_json}

<!-- @format -->

#### Pull your favorite company's CIK from the dictionary 

In [7]:
ticker_symbol = "JPM-PD"   # Because everyone loves Jamie Dimon

company_cik = ticker_cik[ticker_symbol]

<!-- @format -->

#### And go get the list of recent submissions for that company


In [8]:
r.url = f"""{getenv("DATA_URL")}/api/xbrl/companyfacts/CIK{company_cik}.json""" 

req = s.send(r.prepare())

<!-- @format -->

#### Save the results to a file again


In [9]:
with open(file=f"company-facts-{ticker_symbol}.json", mode="w") as file:
    dump(req.json(), file)



#### save all of the facts to a couple of dicts for later use


In [10]:
company_facts: dict = req.json()


#### Go get the revenue and income information for the company





In [11]:
r.url = f"""{getenv("DATA_URL")}/api/xbrl/companyconcept/CIK{company_cik}/us-gaap/Revenues.json"""   # remember that we already have the headers set up

req = s.send(r.prepare())

the_json = req.json()

company_revenue:dict = req.json()

with open(file=f"company-fact-Revenues-{ticker_symbol}.json", mode="w") as file:
    dump(company_revenue, file)

r.url = f"""{getenv("DATA_URL")}/api/xbrl/companyconcept/CIK{company_cik}/us-gaap/NetIncomeLoss.json"""   # remember that we already have the headers set up

req = s.send(r.prepare())

the_json = req.json()

company_NetIncomeLoss:dict = req.json()

with open(file=f"company-fact-NetIncomeLoss-{ticker_symbol}.json", mode="w") as file:
    dump(company_NetIncomeLoss, file)


